In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib


In [8]:
# Load data
data = pd.read_csv('historical_data.csv')

# Display first few rows
print(data.head())

# Handle missing values if any
data = data.dropna()

# Convert appointment_date to datetime
data['appointment_date'] = pd.to_datetime(data['appointment_date'])

# Feature selection and engineering
data['day_of_week'] = data['appointment_date'].dt.dayofweek
data['month'] = data['appointment_date'].dt.month
data['year'] = data['appointment_date'].dt.year

# Select features and target
X = data[['day_of_week', 'month', 'year']]
y = data['absence']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


   appointment_id  patient_id appointment_date  absence
0               1           9       2023-10-23        0
1               2          10       2020-06-07        1
2               3          14       2021-03-12        0
3               4           2       2022-12-30        0
4               5          12       2023-11-05        1


In [9]:
# Initialize the model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

# Save the model and scaler
joblib.dump(model, 'absence_predictor_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


Accuracy: 0.5
Confusion Matrix:
[[2 2]
 [3 3]]
Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.50      0.44         4
           1       0.60      0.50      0.55         6

    accuracy                           0.50        10
   macro avg       0.50      0.50      0.49        10
weighted avg       0.52      0.50      0.51        10



['scaler.pkl']

In [2]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Load the model and scaler
model = joblib.load('absence_predictor_model.pkl')
scaler = joblib.load('scaler.pkl')

@app.route('/predict', methods=['POST'])
def predict_absence():
    data = request.get_json()
    day_of_week = data['day_of_week']
    month = data['month']
    year = data['year']
    
    # Preprocess input data
    input_data = scaler.transform([[day_of_week, month, year]])
    
    # Predict
    prediction = model.predict(input_data)
    
    return jsonify({'absence': int(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1